
## IC-SHM Damage Recognition Model
Run the following cells till the first checkpoint for training

In [ ]:
#Importing the necessary libraries
import platform
print("Python version:", platform.python_version())

import sys
import os

import tensorflow as tf
print("tensorflow version:",tf.__version__)

import numpy as np
print("numpy version:",np.__version__)

import cv2
print("cv2 version:",cv2.__version__)

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
print("matplotlib version:",matplotlib.__version__)

import pandas as pd
print("pandas version:",pd.__version__)

from skimage.transform import resize

from tensorflow.keras import datasets, layers, models

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
#Monting G-Drive to access the datasets
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
#Function to change the path format 
def path_correct(path):
  path = '/'.join(path[2:].split('\\'))
  path = os.path.join(path_ds, path)
  return path

In [ ]:
#function to put the label images into 3 channels
def get_label(file_path):
  
    mask = np.squeeze(file_path, axis = 2)
    target_array = np.zeros((mask.shape[0],mask.shape[1],3))
    target_array[:,:,0]=np.where(mask == 1, 1, 0)
    target_array[:,:,1]=np.where(mask == 2, 1, 0)
    target_array[:,:,2]=np.where(mask == 3, 1, 0)
    
    return target_array

In [ ]:
#function to normalize the images
def normalize(input_image):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  return input_image

In [ ]:
# Functions to plot the images and labels
def show_img(image):
  plt.figure()
  plt.imshow(image)
  plt.axis('off')

def show_label(label):
  fig, axes = plt.subplots(1, 3, figsize=(16, 112))
  y1 = label[:,:,0]
  y2 = label[:,:,1]
  y3 = label[:,:,2]
  plt.axis('off')
  axes[0].axis('off')
  axes[1].axis('off')
  axes[2].axis('off')
  axes[0].imshow(y1)
  axes[1].imshow(y2)
  axes[2].imshow(y3)

In [ ]:
# Function to print the prediction images
def show_pred(y):
  fig, axes = plt.subplots(1, 3, figsize=(16, 112))
  y1 = y[:,:,0]
  y2 = y[:,:,1]
  y3 = y[:,:,2]
  plt.axis('off')
  axes[0].axis('off')
  axes[1].axis('off')
  axes[2].axis('off')
  axes[0].imshow(y1)
  axes[1].imshow(y2)
  axes[2].imshow(y3)

###Checkpoint - 1
Run untill here for training and testing both.

In [ ]:
# Image and Label augmentation function 
def image_augmentation(img):
  seed = (2,3)
  img = tf.image.stateless_random_flip_left_right(img, seed) 
  return img

def label_augmentation(img):
  seed = (2,3)
  img = tf.image.stateless_random_flip_left_right(img, seed)
  return img

In [ ]:
# Preprocess function to run all the required preprocessing functions on the images
def preprocess(x, y):
    def f(x, y):
        x = tf.io.read_file(x)
        y = tf.io.read_file(y)
        x = tf.io.decode_png(x, channels = 3)
        x = tf.image.resize_with_pad(x, 256, 448)
        x = tf.numpy_function(normalize, [x], [tf.float32])
        y = tf.io.decode_bmp(y, channels = 0)
        y = tf.numpy_function(get_label, [y], [tf.float32])
        y = tf.image.resize_with_pad(y, 256, 448)
        x = tf.numpy_function(image_augmentation, [x], [tf.float32])
        y = tf.numpy_function(label_augmentation, [y], [tf.float32])
        
        return x, y

    images, masks = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    images.set_shape([256, 448, 3])
    masks.set_shape([256, 448, 3])

    return images, masks

In [ ]:
# data pipeline function to cache dataset to ram for reducing the data loading bottleneck during training
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=3000)
    dataset = dataset.map(preprocess, num_parallel_calls= tf.data.AUTOTUNE)
    dataset = dataset.batch(batch, num_parallel_calls= tf.data.AUTOTUNE)
    dataset = dataset.cache()
    return dataset

In [ ]:
#Loading the training data for damage images

#Access the csv file containing the absolute directory paths to each file
path_ds = os.path.join('/content/drive/MyDrive/IC-SHM 2021','Tokaido_dataset') #put a path to the dataset
col_names = ['image file name', 'component label file name', 'damage label file name', 'depth image file name', 
             'camera focal length in mm', 'regular images', 'images containing damage in the RRDR']
ftrain = pd.read_csv(os.path.join(path_ds,'files_train.csv'),names = col_names,delimiter=',')
ftrain.iloc[:,0] = ftrain.iloc[:,0].apply(lambda x: path_correct(x))
ftrain.iloc[:,1] = ftrain.iloc[:,1].apply(lambda x: path_correct(x))
ftrain.iloc[:,2] = ftrain.iloc[:,2].apply(lambda x: path_correct(x))
ftrain.iloc[:,3] = ftrain.iloc[:,3].apply(lambda x: path_correct(x))
train_comp = ftrain.loc[ftrain['regular images']==True, ['image file name', 'component label file name', 'damage label file name', 'depth image file name', 'camera focal length in mm']]
train_dmg = ftrain.loc[ftrain['images containing damage in the RRDR']==True, ['image file name', 'component label file name', 'damage label file name', 'depth image file name', 'camera focal length in mm']]


In [ ]:
#splitting the data for training and validation

#Run for training on the damage images
train1 = train_dmg[:4000]
val1 = train_dmg[4000:]


In [ ]:
# Creating the tf dataset from dataframe containing image addresses

images = train1['image file name'].to_numpy()
masks = train1['damage label file name'].to_numpy()
dataset = tf_dataset(images, masks)
images_val = val1['image file name'].to_numpy()
masks_val = val1['damage label file name'].to_numpy()
dataset_val = tf_dataset(images_val, masks_val)

In [ ]:
#Printing out an image for reference
for x, y in dataset.take(1):
        show_img(x[0])
        show_label(y[0])
        
        break

###Checkpoint - 2

In [ ]:
#Importing the necessary Keras libraries for Creating the U-Net Model with gated attention and residual connections

from keras.layers import Conv2D, BatchNormalization, Activation, Add, Dropout, UpSampling2D, Input, Multiply, MaxPooling2D, Concatenate
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras import backend as K

#number of features to be used in the model df for encoder and uf for decoder, 32 was found to be optimal for both
df=32
uf=32
    
def build_unet(input_shape):

  #convolution block with batch normalization to be used in both encoder and decoder

  def conv2d(layer_input,filters,dropout_rate=0):
    d = Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(layer_input)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)
      
    d = Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(d)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)

    d = Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(d)
    d = BatchNormalization()(d)

    sc = Conv2D(filters,kernel_size=(1,1),padding='same')(layer_input)
    sc = BatchNormalization()(sc)

    rp = Add()([sc,d])
    rp = Activation('relu')(rp)
    rp = Dropout(dropout_rate)(rp)
      
    return rp

  #convolution block without batch normalization to be used in both encoder and decoder

  def conv2d_wobn(layer_input,filters,dropout_rate=0):
    d = Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(layer_input)
    d = Activation('relu')(d)
      
    d = Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(d)
    d = Activation('relu')(d)

    d = Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(d)

    sc = Conv2D(filters,kernel_size=(1,1),padding='same')(layer_input)

    rp = Add()([sc,d])
    rp = Activation('relu')(rp)
    rp = Dropout(dropout_rate)(rp)

    return rp
    
  #upsampling and convolution block with batch normalization to be used in decoder

  def deconv2d(layer_input,filters):
    u = UpSampling2D((2,2))(layer_input)
    u = Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(u)
    u = BatchNormalization()(u)
    u = Activation('relu')(u)

    u = Conv2D(filters,kernel_size=(3,3),strides=(1,1),padding='same')(u)
    u = BatchNormalization()(u)
    u = Activation('relu')(u)
      
    return u

  #attention block with batch normalization to be used in place of skip connections for gated attention

  def attention_block(F_g,F_l,F_int):
    g = Conv2D(F_int,kernel_size=(1,1),strides=(1,1),padding='valid')(F_g)
    g = BatchNormalization()(g)
    x = Conv2D(F_int,kernel_size=(1,1),strides=(1,1),padding='valid')(F_l)
    x = BatchNormalization()(x)
    psi = Add()([g,x])
    psi = Activation('relu')(psi)
    
    psi = Conv2D(1,kernel_size=(1,1),strides=(1,1),padding='valid')(psi)
      
    psi = BatchNormalization()(psi)
    psi = Activation('sigmoid')(psi)
      
    return Multiply()([F_l,psi])

  #actual model definition depth = 4, i.e. 4 times downsampled by fator of 2 (by maxpooling)

  inputs = Input(input_shape)
    
  conv1 = conv2d_wobn(inputs,df)
  pool1 = MaxPooling2D((2,2))(conv1)
    
  conv2 = conv2d(pool1,df*2)
  pool2 = MaxPooling2D((2,2))(conv2)
    
  conv3 = conv2d(pool2,df*4)
  pool3 = MaxPooling2D((2,2))(conv3)
    
  conv4 = conv2d(pool3,df*8)
  pool4 = MaxPooling2D((2,2))(conv4)
  
  conv5 = conv2d(pool4,df*8)
    
  up6 = deconv2d(conv5,uf*8)
  conv6 = attention_block(up6,conv4,uf*8)
  up6 = Concatenate()([up6,conv6])
  conv6 = conv2d_wobn(up6,uf*8)
  
  up7 = deconv2d(conv6,uf*4)
  conv7 = attention_block(up7,conv3,uf*4)
  up7 = Concatenate()([up7,conv7])
  conv7 = conv2d_wobn(up7,uf*4)
    
  up8 = deconv2d(conv7,uf*2)
  conv8 = attention_block(up8,conv2,uf*2)
  up8 = Concatenate()([up8,conv8])
  conv8 = conv2d_wobn(up8,uf*2)
    
  up9 = deconv2d(conv8,uf)
  conv9 = attention_block(up9,conv1,uf)
  up9 = Concatenate()([up9,conv9])
  conv9 = conv2d_wobn(up9,uf)

  #here the first input to Conv2D i.e. 3 defines the output number of classes to be pridicted

  outputs = Conv2D(3,kernel_size=(1,1),strides=(1,1),activation='sigmoid')(conv9)
   
  model = Model(inputs=inputs,outputs=outputs)
    
  return model

In [ ]:
#building the unet model with gated attention and residual connections and printing its summary
model = build_unet((256,448,3))
model.summary()

###Checkpoint 3



In [ ]:
#using Adam optimizer and fixing the learning rate 
optimizer_adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

In [ ]:
#Checkpoint for model saving and reloading the saved weights
# "training_dmg_resatt_final/cp_dmg.ckpt" this checkpoint to be supplied for best weights as trained by us
checkpoint_path = os.path.join(path_ds, "training_dmg_resatt_final/cp_dmg.ckpt")
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,  monitor = 'iou', mode='max', save_best_only = True,
                                                 verbose=1)

In [ ]:
""" Courtesy stackoverflow Daniel Möller https://stackoverflow.com/users/2097240/daniel-m%c3%b6ller?tab=profile
answer to question 'Custom loss function for U-net in keras using class weights: `class_weight` not supported for 3+ dimensional targets' 
"""
# Weighted loss function 
def weightedLoss(originalLossFunc, weightsList):

    def lossFunc(true, pred):

        axis = -1 #if channels last 
        #axis=  1 #if channels first


        #argmax returns the index of the element with the greatest value
        #done in the class axis, it returns the class index    
        classSelectors = K.argmax(true, axis=axis) 
            #if your loss is sparse, use only true as classSelectors

        #considering weights are ordered by class, for each class
        #true(1) if the class index is equal to the weight index   
        classSelectors = [K.equal(i, classSelectors) for i in range(len(weightsList))]

        #casting boolean to float for calculations  
        #each tensor in the list contains 1 where ground true class is equal to its index 
        #if you sum all these, you will get a tensor full of ones. 
        classSelectors = [K.cast(x, K.floatx()) for x in classSelectors]

        #for each of the selections above, multiply their respective weight
        weights = [sel * w for sel,w in zip(classSelectors, weightsList)] 

        #sums all the selections
        #result is a tensor with the respective weight for each element in predictions
        weightMultiplier = weights[0]
        for i in range(1, len(weights)):
            weightMultiplier = weightMultiplier + weights[i]


        #make sure your originalLossFunc only collapses the class axis
        #you need the other axes intact to multiply the weights tensor
        loss = originalLossFunc(true,pred) 
        loss = loss * weightMultiplier

        return loss
    return lossFunc

In [ ]:
#summation of dice loss and cross entropy loss

from keras import backend as K
def DiceCELoss(targets, inputs, smooth=1e-6):
    CE = tf.keras.metrics.binary_crossentropy(targets, inputs)
    y_true_f=K.flatten(targets)
    y_pred_f=K.flatten(inputs)
    intersection=K.sum(y_true_f*y_pred_f)
    dice_loss=1-((2*intersection) + smooth)/(K.sum(y_true_f*y_true_f)+K.sum(y_pred_f*y_pred_f)+ smooth)
    dice_CE=dice_loss+CE
    return dice_CE

In [ ]:
#summation of log of (1-dice loss) and cross entropy loss

import math
from keras import backend as K
def logDiceCELoss(targets, inputs, smooth=1e-6):
    CE = tf.keras.metrics.binary_crossentropy(targets, inputs)
    #CE = tf.keras.metrics.CategoricalCrossentropy(targets, inputs)
    y_true_f=K.flatten(targets)
    y_pred_f=K.flatten(inputs)
    intersection=K.sum(y_true_f*y_pred_f)
    dice_loss=1-((2*intersection) + smooth)/(K.sum(y_true_f*y_true_f)+K.sum(y_pred_f*y_pred_f)+ smooth)
    dice_CE=CE-math.log(1-dice_loss)
    return dice_CE

In [ ]:
# The weights for each class are hardcoded here and are calculated using the code presented towards the end of this notebook
w1= 0.03611688490088685
w2= 1.0
w3=9.345573778790564

In [ ]:
#best results were obtained with binary cross entropy loss 

model.compile(optimizer= optimizer_adam,
              loss= weightedLoss(tf.keras.losses.BinaryCrossentropy(), [w1,w2,w3]),
              metrics=[
                       tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.Recall(thresholds = 0.2),
                       tf.keras.metrics.Precision(thresholds = 0.7),
                       tf.keras.metrics.MeanIoU(3,name="iou")])

In [ ]:
#run this cell for loading the weights from saved checkpoints
model.load_weights(checkpoint_path)

###Checkpoint 4

In [ ]:
#Train the model
history = model.fit(dataset, epochs= 10 ,validation_data= dataset_val, verbose = 1, callbacks=[cp_callback])

In [ ]:
#Print Loss
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure()
plt.plot(history.epoch, loss, 'r', label='Training loss')
plt.plot(history.epoch, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.legend()
plt.show()

In [ ]:
#Print Binary Accuracy
accu = history.history['binary_accuracy']
val_accu = history.history['val_binary_accuracy']

plt.figure()
plt.plot(history.epoch, accu, 'r', label='Training binary accuracy')
plt.plot(history.epoch, val_accu, 'b', label='Validation binary accuracy')
plt.title('Training and Validation binary accuracy')
plt.xlabel('Epoch')
plt.ylabel('binary accuracy Value')
plt.legend()
plt.show()

In [ ]:
#print recall
recall = history.history['recall']
val_recall = history.history['val_recall']

plt.figure()
plt.plot(history.epoch, recall, 'r', label='Training recall')
plt.plot(history.epoch, val_recall, 'b', label='Validation recall')
plt.title('Training and Validation recall')
plt.xlabel('Epoch')
plt.ylabel('recall Value')
plt.legend()
plt.show()

In [ ]:
#Print precision
pres = history.history['precision']
val_pres = history.history['val_precision']

plt.figure()
plt.plot(history.epoch, pres, 'r', label='Training precision')
plt.plot(history.epoch, val_pres, 'b', label='Validation precision')
plt.title('Training and Validation precision')
plt.xlabel('Epoch')
plt.ylabel('precision Value')
plt.legend()
plt.show()

In [ ]:
#Print Mean IoU
miou = history.history['iou']
val_miou = history.history['val_iou']

plt.figure()
plt.plot(history.epoch, miou, 'r', label='Training mean_io_u')
plt.plot(history.epoch, val_miou, 'b', label='Validation mean_io_u')
plt.title('Training and Validation mean_io_u')
plt.xlabel('Epoch')
plt.ylabel('mean_io_u Value')
plt.legend()
plt.show()

In [ ]:
#Evaluate the model on validation dataset
model.evaluate(dataset_val, verbose = 1)

In [ ]:
#Printing predictions on the validation dataset
z=1
for x, y in dataset_val.take(10):
        z += 1
        show_img(x[2])
        show_label(y[2])
        pred = model.predict(x)
        show_pred(pred[2])
        if (z==9):
          break

###Checkpoint 5

In [ ]:
#Preprocessing for the test dataset
def preprocess_test(x,y):
    def f(x,y):
        x = tf.io.read_file(x)
        y = tf.io.read_file(y)
        x = tf.io.decode_png(x, channels = 3)
        x = tf.image.resize_with_pad(x, 256,448)
        x = tf.numpy_function(normalize, [x], [tf.float32]) 
        y = tf.io.decode_png(y, channels = 0)   
        y = tf.numpy_function(get_label_tex, [y], [tf.float32])
        y = tf.image.resize_with_pad(y, 256, 448)   

        return x, y

    images, masks = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    images.set_shape([256, 448, 3])
    masks.set_shape([256, 448, 3])

    return images, masks

In [ ]:
#dataloader for the test dataset
def tf_dataset_test(x,y, batch = 8):
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.map(preprocess_test, num_parallel_calls= tf.data.AUTOTUNE)
    dataset = dataset.batch(batch, num_parallel_calls= tf.data.AUTOTUNE)
    dataset = dataset.cache()
    return dataset

In [ ]:
#Function to change the path format 
def path_correct_png(path):
  path = '/'.join(path[2:].split('\\'))
  path = os.path.join(path_ds, path)
  path = path[:-4] + '.png'
  return path

In [ ]:
# Loading the training data from the component demage test dataset file

#Access the csv file containing the absolute directory paths to each file
path_ds = os.path.join('/content/drive/MyDrive/IC-SHM 2021','Tokaido_dataset') #put a path to the dataset
col_names = ['image file name', 'component label file name', 'damage label file name', 'depth image file name', 
             'camera focal length in mm', 'regular images', 'images containing damage in the RRDR']
ftest = pd.read_csv(os.path.join(path_ds,'files_test.csv'),names = col_names,delimiter=',')
ftest.iloc[:,0] = ftest.iloc[:,0].apply(lambda x: path_correct(x))
ftest.iloc[:,1] = ftest.iloc[:,1].apply(lambda x: path_correct(x))
ftest.iloc[:,2] = ftest.iloc[:,2].apply(lambda x: path_correct_png(x))
ftest.iloc[:,3] = ftest.iloc[:,3].apply(lambda x: path_correct(x))
test_comp = ftest.loc[ftest['images containing damage in the RRDR']==True, ['image file name', 'component label file name', 'damage label file name', 'depth image file name', 'camera focal length in mm']]


In [ ]:
# creating dataloader for component damage images

images_test = test_comp['image file name'].to_numpy()
masks_test = test_comp['damage label file name'].to_numpy()
dataset_test = tf_dataset_test(images_test,masks_test)
address = test_comp['damage label file name'].apply(lambda x : os.path.split(x)[1]).to_numpy()

In [ ]:
#Printing predictions on the component damage test dataset
z=1
for x, y in dataset_test.take(10):
        z += 1
        show_img(x[0])
        show_label(y[0])
        pred = model.predict(x)
        show_pred(pred[0])
        if (z==9):
          break

###Checkpoint 6

In [ ]:
#Function to create the predictions remove the padding 2 pixels from top and 2 pixels from bottom and upscale them to size (360, 640) and then save in the drive folder
#this cell saves pridiction on damage component test images
from PIL import Image
import imageio

root_path = '/content/drive/MyDrive/IC-SHM 2021/Tokaido_dataset/Prediction_lab'

def size_changer(img):
  mask = np.zeros((360,640, img.shape[2]), dtype = np.uint8)
  temp = np.where(img[:,:,0] > 0.5, 1, 0)
  temp = cv2.resize(temp.astype('uint8'), dsize = (640,360), interpolation=cv2.INTER_CUBIC)
  temp = np.where(temp > 0.5, 1, 0)
  mask[:,:,0] = temp

  temp = np.where(img[:,:,1] > 0.5, 1, 0)
  temp = cv2.resize(temp.astype('uint8'), dsize = (640,360), interpolation=cv2.INTER_CUBIC)
  temp = np.where(temp > 0.5, 2, 0)
  mask[:,:,1] = temp

  temp = np.where(img[:,:,2] > 0.5, 1, 0)
  temp = cv2.resize(temp.astype('uint8'), dsize = (640,360), interpolation=cv2.INTER_CUBIC)
  temp = np.where(temp > 0.5, 3, 0)
  mask[:,:,2] = temp

  return mask

def crop(img):
  img=img[2:254,:,:]
  return img

def mask_save(img, save_path):
  path = os.path.join(root_path, 'synthetic/test/labdmg')
  img=crop(img)
  img = size_changer(img)
  mask = img

  output = np.zeros((mask.shape[0],mask.shape[1]), dtype = np.uint8)
  output = np.argmax(mask, axis = 2)
  output = output + 1

  imageio.imwrite(os.path.join(path, save_path), output.astype('uint8'))

  return None

i = 0
for item in dataset_test:
  p = model.predict(item[0])
  for j in range(len(p)):
    mask_save(p[j], os.path.join(address[i]))
    i += 1
    print(i)

In [ ]:
#Function to create the predictions remove the padding 2 pixels from top and 2 pixels from bottom and upscale them to size (360, 640) and then save in the drive folder
#this cell saves pridiction on texture test images
from PIL import Image
import imageio

root_path = '/content/drive/MyDrive/IC-SHM 2021/Tokaido_dataset/Prediction_lab'

def size_changer(img):
  mask = np.zeros((360,640, img.shape[2]), dtype = np.uint8)

  mask = np.zeros((360,640, img.shape[2]), dtype = np.uint8)
  temp = np.where(img[:,:,0] > 0.5, 1, 0)
  temp = cv2.resize(temp.astype('uint8'), dsize = (640,360), interpolation=cv2.INTER_CUBIC)
  temp = np.where(temp > 0.5, 1, 0)
  mask[:,:,0] = temp

  temp = np.where(img[:,:,1] > 0.4, 1, 0)
  temp = cv2.resize(temp.astype('uint8'), dsize = (640,360), interpolation=cv2.INTER_CUBIC)
  temp = np.where(temp > 0.5, 2, 0)
  mask[:,:,1] = temp

  temp = np.where(img[:,:,2] > 0.4, 1, 0)
  temp = cv2.resize(temp.astype('uint8'), dsize = (640,360), interpolation=cv2.INTER_CUBIC)
  temp = np.where(temp > 0.5, 3, 0)
  mask[:,:,2] = temp

  return mask

def crop(img):
  img=img[2:254,:,:]
  return img

def mask_save(img, save_path):
  path = os.path.join(root_path, 'synthetic_puretex/labdmg')
  img=crop(img)
  img = size_changer(img)
  mask = img

  output = np.zeros((mask.shape[0],mask.shape[1]), dtype = np.uint8)
  output = np.argmax(mask, axis = 2)
  output = output + 1
  
  imageio.imwrite(os.path.join(path, save_path), output.astype('uint8'))
  return None

i = 0
for item in dataset_test_tex:
  p = model.predict(item[0])
  for j in range(len(p)):
    mask_save(p[j], os.path.join(address_tex[i]))
    i += 1
    print(i)

In [ ]:
#Code to find the percentage of pixels of each class 
import skimage.io
p_of_class_1_pix=0
p_of_class_2_pix=0
p_of_class_3_pix=0

for i in range(len(masks)):
  img = skimage.io.imread(masks[i])
  #plt.imshow(img)
  #print(img.shape)
  
  # counting the number of pixels
  p_of_class_1_pix = p_of_class_1_pix+((np.sum(img == 1)/(360*640))*100)
  p_of_class_2_pix = p_of_class_2_pix+((np.sum(img == 2)/(360*640))*100)
  p_of_class_3_pix = p_of_class_3_pix+((np.sum(img == 3)/(360*640))*100)


p_of_class_1_pix=p_of_class_1_pix/len(masks)
p_of_class_2_pix=p_of_class_2_pix/len(masks)
p_of_class_3_pix=p_of_class_3_pix/len(masks)


print('Number of class 1 pixels:', p_of_class_1_pix)
print('Number of class 2 pixels:', p_of_class_2_pix)
print('Number of class 3 pixels:', p_of_class_3_pix)

In [ ]:
#Code to find the weights needed for the weighted loss function
#for median frequency weighting percentage of class 2 is median
w1 = p_of_class_2_pix/p_of_class_1_pix
w2 = p_of_class_2_pix/p_of_class_2_pix
w3 = p_of_class_2_pix/p_of_class_3_pix

print(w1,w2,w3)